<a id="sectionx"></a>
# MPO FD Rollover Testing - Notebook
* [Get FDs Maturing today](#section1)
* [Create new Rollover Term Deposit - Monthly (manual)](#section2)
* [Create new Rollover Term Deposit - Monthly (in one step)](#section2b)



GitHub: https://github.com/MkershMambu/MambuAPINotebook/blob/master/MPO%20FD%20Rollover%20Test.ipynb

Viewer: [https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/MPO%20FD%20Rollover%20Test.ipynb](https://nbviewer.jupyter.org/github/MkershMambu/MambuAPINotebook/blob/master/MPO%20FD%20Rollover%20Test.ipynb?flush_cache=true)



In [1]:
%run API.py
%run ENV.py
from IPython.core.display import HTML
setENV(ENV)

<a id="section1"></a>
[back](#sectionx)
### API call to determine FD's maturing today
* This was in the original spec doc but isn't the call that is being made
* For demo purposes though it may make sense to use this filter

In [29]:
%%writefile JSONBODY
{
  "filterConstraints": [
      {
          "filterSelection": "MATURITY_DATE",
          "filterElement": "TODAY"
      },
        {
            "filterElement": "IN",
            "filterSelection": "ACCOUNT_STATE",
            "values": [
                "ACTIVE"
            ]
        }
  ]
}

Overwriting JSONBODY


In [30]:
HEADERS = {'Content-Type': 'application/json'}
PARAMS = {}
r = POST('{{env1}}/savings/search', headers=HEADERS, params=PARAMS, body="JSONBODY")
print(len(r.json()))
PRINT(r)

1
API status: 200
JSON Response:
[
    {
        "accountHolderKey": "8a8186016a73c831016a788d7fa32d2e",
        "accountHolderType": "CLIENT",
        "accountState": "ACTIVE",
        "accountType": "FIXED_DEPOSIT",
        "accruedInterest": "0",
        "activationDate": "2018-06-21T00:00:00+0000",
        "allowOverdraft": false,
        "approvedDate": "2019-06-21T17:22:40+0000",
        "assignedBranchKey": "8a8186016a73c831016a77fe7fcf2b21",
        "availableBalance": "5571.58",
        "balance": "5571.58",
        "creationDate": "2019-06-21T15:22:32+0000",
        "currency": {
            "code": "EUR",
            "creationDate": "2018-12-05T14:25:48+0000",
            "currencySymbolPosition": "BEFORE_NUMBER",
            "digitsAfterDecimal": 2,
            "isBaseCurrency": true,
            "lastModifiedDate": "2018-12-05T14:25:48+0000",
            "name": "Euro",
            "symbol": "\u20ac"
        },
        "currencyCode": "EUR",
        "encodedKey": "8a8187a1

### The API that the Rollover App was using
* NOTE: Need EOD to run before FD(s) moved into MATURED state
    * So not great for demos
* Best option in a live PROD enviornment though

In [17]:
%%writefile JSONBODY1
{
  "filterConstraints": [
      {
          "filterElement": "IN",
          "filterSelection": "ACCOUNT_STATE",
          "values": [
              "MATURED"
          ]
          
      }
  ]
}

Overwriting JSONBODY1


In [18]:
HEADERS = {'Content-Type': 'application/json'}
PARAMS = {}
r = POST('{{env1}}/savings/search', headers=HEADERS, params=PARAMS, body="JSONBODY1")
print(len(r.json()))
PRINT(r)

1
API status: 200
JSON Response:
[
    {
        "accountHolderKey": "8a81866d67cfe3f20167cffd9fc50024",
        "accountHolderType": "CLIENT",
        "accountState": "MATURED",
        "accountType": "FIXED_DEPOSIT",
        "accruedInterest": "0",
        "activationDate": "2018-12-21T10:02:48+0000",
        "allowOverdraft": false,
        "approvedDate": "2018-12-21T10:02:00+0000",
        "assignedBranchKey": "8a81872d6787e9f30167a344a74e4e1c",
        "availableBalance": "10274.25",
        "balance": "10274.25",
        "creationDate": "2018-12-21T09:01:54+0000",
        "currency": {
            "code": "EUR",
            "creationDate": "2018-12-05T14:25:48+0000",
            "currencySymbolPosition": "BEFORE_NUMBER",
            "digitsAfterDecimal": 2,
            "isBaseCurrency": true,
            "lastModifiedDate": "2018-12-05T14:25:48+0000",
            "name": "Euro",
            "symbol": "\u20ac"
        },
        "currencyCode": "EUR",
        "encodedKey": "8a818

<a id="section2"></a>
[back](#sectionx)
### Create new Rollover Term Deposit - Monthly (manual)
* With maturity of today so that it triggers the Rollover App
* This section takes you through step-by-step
   * To perform via one function call see XXX

In [21]:
%%writefile JSONBODY2
{
  "accountType": "FIXED_DEPOSIT",
  "name": "FD Rollover - Monthly",
  "accountHolderKey": "8a8186016a73c831016a788d7fa32d2e",
  "productTypeKey": "8a8186016a73c831016a7807c0bb2b2a",
  "currencyCode": "EUR",
  "accountHolderType": "CLIENT",
  "interestSettings": {
      "interestRateSettings": {
            "encodedKey": "8a8186016a7a9872016a7e8d7e7e179e",
            "interestChargeFrequency": "ANNUALIZED",
            "interestChargeFrequencyCount": 1,
            "interestRate": 2.54,
            "interestRateTerms": "FIXED",
            "interestRateTiers": []
        }
    }
}


Overwriting JSONBODY2


In [22]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
r = POST('{{env1}}/deposits', headers=HEADERS, params=PARAMS, body="JSONBODY2")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)
encodedKey = r.json()["encodedKey"]

API status:
201
Now the JSON:
{"encodedKey":"8a8187a16b7a71dc016b7a9e0dd40382","creationDate":"2019-06-21T17:22:32+02:00","lastModifiedDate":"2019-06-21T17:22:32+02:00","id":"BVTA788","name":"FD Rollover - Monthly","accountHolderType":"CLIENT","accountHolderKey":"8a8186016a73c831016a788d7fa32d2e","accountState":"PENDING_APPROVAL","productTypeKey":"8a8186016a73c831016a7807c0bb2b2a","accountType":"FIXED_DEPOSIT","currencyCode":"EUR","assignedBranchKey":"8a8186016a73c831016a77fe7fcf2b21","internalControls":{},"overdraftSettings":{"allowOverdraft":false,"overdraftLimit":0},"interestSettings":{"interestRateSettings":{"encodedKey":"8a8187a16b7a71dc016b7a9e0dd40383","interestRate":2.54,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED"},"interestPaymentSettings":{"interestPaymentPoint":"EVERY_MONTH","interestPaymentDates":[]}},"overdraftInterestSettings":{},"balances":{"overdraftAmount":0,"technicalOverdraftAmount":0,"loc

### Approve the account

In [23]:
%%writefile JSONBODY3
{
  "action": "APPROVE",
  "notes": "more notes"
}

Overwriting JSONBODY3


In [24]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY3")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a8187a16b7a71dc016b7a9e0dd40382","creationDate":"2019-06-21T17:22:32+02:00","lastModifiedDate":"2019-06-21T17:22:39+02:00","id":"BVTA788","name":"FD Rollover - Monthly","accountHolderType":"CLIENT","accountHolderKey":"8a8186016a73c831016a788d7fa32d2e","accountState":"APPROVED","productTypeKey":"8a8186016a73c831016a7807c0bb2b2a","accountType":"FIXED_DEPOSIT","approvedDate":"2019-06-21T17:22:40+02:00","currencyCode":"EUR","assignedBranchKey":"8a8186016a73c831016a77fe7fcf2b21","internalControls":{},"overdraftSettings":{"allowOverdraft":false,"overdraftLimit":0E-10},"interestSettings":{"interestRateSettings":{"encodedKey":"8a8187a16b7a71dc016b7a9e0dd40383","interestRate":2.54000000000000000000,"interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateTiers":[],"interestRateTerms":"FIXED"},"interestPaymentSettings":{"interestPaymentPoint":"EVERY_MONTH","interestPaymentDates":[]}},"overdraftInterestSettings":{},"balance

### Deposit into account

In [25]:
%%writefile JSONBODY4
{
  "transactionDetails": {
    "transactionChannelId": "8a818e74677a2e9201677ec2b4c336a6"
  },
  "amount": 5432,
  "notes": "some notes",
  "paymentOrderId": "{{UniqueID}}",
  "externalId": "{{UniqueID}}",
  "valueDate": "2018-06-21T00:00:00+02:00"
}

Overwriting JSONBODY4


In [26]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
BODYPARTS = {'UniqueID': getUUID()}
url = "{{env1}}" + "/deposits/{0}/deposit-transactions".format(encodedKey)
r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY4", bodyparts=BODYPARTS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
201
Now the JSON:
{"encodedKey":"8a8187a16b7a71dc016b7aa20b690389","id":"275","externalId":"1b61a348-44e3-4899-961d-3034c4d4e73f","paymentOrderId":"1b61a348-44e3-4899-961d-3034c4d4e73f","creationDate":"2019-06-21T17:22:48+02:00","valueDate":"2018-06-21T00:00:00+02:00","notes":"some notes","parentAccountKey":"8a8187a16b7a71dc016b7a9e0dd40382","type":"DEPOSIT","amount":5432,"currencyCode":"EUR","affectedAmounts":{"fundsAmount":5432.0000000000,"interestAmount":0,"feesAmount":0,"overdraftAmount":0E-10,"overdraftFeesAmount":0E-10,"overdraftInterestAmount":0E-10,"technicalOverdraftAmount":0E-10,"technicalOverdraftInterestAmount":0E-10,"fractionAmount":0},"taxes":{},"accountBalances":{"totalBalance":5432.0000000000},"userKey":"8a8186ac69267891016928db604e0735","branchKey":"8a8186016a73c831016a77fe7fcf2b21","terms":{"interestSettings":{"interestRate":2.54000000000000000000},"overdraftInterestSettings":{},"overdraftSettings":{}},"transactionDetails":{"transactionChannelKey":"8a818e7

### Start the maturity

In [27]:
%%writefile JSONBODY5
{
  "type": "START_MATURITY",
  "notes": "start maturity for FD"
}

Overwriting JSONBODY5


In [28]:
HEADERS = {'Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/savings/{0}/transactions".format(encodedKey)

r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY5")
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
200
Now the JSON:
{"encodedKey":"8a8187a16b7a71dc016b7a9e0dd40382","id":"BVTA788","accountHolderKey":"8a8186016a73c831016a788d7fa32d2e","accountHolderType":"CLIENT","name":"FD Rollover - Monthly","creationDate":"2019-06-21T15:22:32+0000","approvedDate":"2019-06-21T17:22:40+0000","activationDate":"2018-06-21T00:00:00+0000","lastModifiedDate":"2019-06-21T15:22:57+0000","lastInterestCalculationDate":"2019-06-21T00:00:00+0000","lastInterestStoredDate":"2019-06-21T00:00:00+0000","productTypeKey":"8a8186016a73c831016a7807c0bb2b2a","accountType":"FIXED_DEPOSIT","accountState":"ACTIVE","maturityDate":"2019-06-21T00:00:00+0000","balance":"5571.58","accruedInterest":"0","overdraftInterestAccrued":"0","technicalOverdraftInterestAccrued":"0","overdraftAmount":"0","technicalOverdraftAmount":"0","interestSettings":{"interestRate":"2.54","encodedKey":"8a8187a16b7a71dc016b7a9e0dd40383","interestChargeFrequency":"ANNUALIZED","interestChargeFrequencyCount":1,"interestRateSource":"FIXED_INTER

<a id="section2b"></a>
[back](#sectionx)
### Create new Rollover Term Deposit - Monthly (in one step)
* This section will only be relevant if you have an executable version of this notebook

In [172]:
def createFDMonthly_MaturesToday():
    # Create a new FD
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    r = POST('{{env1}}/deposits', headers=HEADERS, params=PARAMS, body="JSONBODY2")
    print ("API status:")
    print(r.status_code)
    print("Now the JSON:")
    print(r.text)
    encodedKey = r.json()["encodedKey"]
    
    # Approve the account
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    url = "{{env1}}" + "/deposits/{0}:changeState".format(encodedKey)
    r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY3")
    
    # Deposit into the account
    HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
    PARAMS = {}
    BODYPARTS = {'UniqueID': getUUID()}
    url = "{{env1}}" + "/deposits/{0}/deposit-transactions".format(encodedKey)
    r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY4", bodyparts=BODYPARTS)
    
    # Start the Maturity
    HEADERS = {'Content-Type': 'application/json'}
    PARAMS = {}
    url = "{{env1}}" + "/savings/{0}/transactions".format(encodedKey)

    r = POST(url, headers=HEADERS, params=PARAMS, body="JSONBODY5")

## Miscellaneous - Helper APIs

### Delete Account

In [14]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json','Content-Type': 'application/json'}
PARAMS = {}
url = "{{env1}}" + "/deposits/{0}".format(encodedKey)
r = DELETE(url, headers=HEADERS, params=PARAMS)
print ("API status:")
print(r.status_code)
print("Now the JSON:")
print(r.text)

API status:
400
Now the JSON:
{"errors":[{"errorCode":105,"errorReason":"INVALID_ACCOUNT_STATE"}]}


### Get account details

In [68]:
HEADERS = {'Accept': 'application/vnd.mambu.v2+json'}
# Parameters below set up pagination and only returns max 20
PARAMS = {'detailsLevel': 'FULL', 'paginationDetails': 'ON','limit': 20}
r = GET('{{env1}}/deposits/APIS515', headers=HEADERS, params=PARAMS)
PRINT(r)

API status: 200
JSON Response:
{
    "accountHolderKey": "8a8186016a73c831016a788d7fa32d2e",
    "accountHolderType": "CLIENT",
    "accountState": "ACTIVE",
    "accountType": "FIXED_DEPOSIT",
    "accruedAmounts": {
        "interestAccrued": 0.4996416603,
        "overdraftInterestAccrued": 0.0,
        "technicalOverdraftInterestAccrued": 0.0
    },
    "activationDate": "2018-05-02T00:00:00+02:00",
    "approvedDate": "2019-05-03T16:30:36+02:00",
    "assignedBranchKey": "8a8186016a73c831016a77fe7fcf2b21",
    "balances": {
        "availableBalance": 7179.89,
        "feesDue": 0.0,
        "holdBalance": 0.0,
        "lockedBalance": 0.0,
        "overdraftAmount": 0.0,
        "overdraftInterestDue": 0.0,
        "technicalOverdraftAmount": 0.0,
        "technicalOverdraftInterestDue": 0.0,
        "totalBalance": 7179.89
    },
    "creationDate": "2019-05-03T16:30:21+02:00",
    "currencyCode": "EUR",
    "encodedKey": "8a8186016a7a9872016a7e19680c0ec5",
    "id": "APIS515",
